In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from tensorflow.contrib import rnn

In [22]:
# Data Collection
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST/", one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [23]:
# Defining the model parameters

# Number of classes to classify the input data
n_classes = 10
# In order to handle the large data size, we will only concentrate on a batch of data
batch_size = 128

# Defining the chunk size and number of chunks for LSTM
chunk_size = 28
n_chunks = 28
rnn_size = 128

# Defining the number of epochs : Cycles of feedforward + backprop
hm_epochs = 10

# Defining the output as well as the input data
x = tf.placeholder('float', [None, n_chunks, chunk_size])
y = tf.placeholder('float')

In [27]:
# Deining the Neural Network Model
def recurrent_neural_network_model(x):
    # Defining the weights for the first hidden layer with the shape no_inputs X n_nodes_h1
    layer = {'weights': tf.Variable(tf.random_normal([rnn_size, n_classes])), 
                     'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1,chunk_size])
    x = tf.split(x, n_chunks, 0)
    
    lstm_cell = rnn.BasicLSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output

In [28]:
# Training the neural network
def train_neural_network(x,y):
    prediction = recurrent_neural_network_model(x)
    # Calculating the cost function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels= y))
    # Optimizing the cost function : Using the AdamOptimizer and the default learning rate of 0.01
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    # Running the tensorflow session
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # Running through the epochs and training the network
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                x_t, y_t = mnist.train.next_batch(batch_size)
                x_t = x_t.reshape((batch_size, n_chunks, chunk_size))
                # Run the optimizer on the cost function
                sess.run(optimizer, feed_dict={x: x_t, y: y_t})
            print("Epoch ", epoch, "completed out of ", hm_epochs)
            
        # Testing the network
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print("Accuracy" , accuracy.eval({x: mnist.test.images.reshape((-1, n_chunks, chunk_size)), y: mnist.test.labels})) 

In [29]:
train_neural_network(x,y)

Epoch  0 completed out of  10
Epoch  1 completed out of  10
Epoch  2 completed out of  10
Epoch  3 completed out of  10
Epoch  4 completed out of  10
Epoch  5 completed out of  10
Epoch  6 completed out of  10
Epoch  7 completed out of  10
Epoch  8 completed out of  10
Epoch  9 completed out of  10
Accuracy 0.9848
